In [ ]:
!pip install yfinance pandas-datareader finance-datareader

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr

In [ ]:
# MF_Trading Rate Back Testing 함수화

def get_return_mf(code):
  df = fdr.DataReader(code,'2018')

  df = df.reset_index()
  df = df[['Date','Open']]

  df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

  cond = (df['Day'] == 'Friday') | (df['Day'] == 'Monday')

  df = df[cond]

  if df.iloc[0,2] == 'Friday':
    df = df.drop(index = df.index[0])

  if df.iloc[-1,2] == 'Monday':
    df = df.drop(index = df.index[-1])

  cond = (df['Day'] == 'Monday') & (df['Day'].shift(-1) == 'Monday')
  df = df.drop(index = df[cond].index)

  cond = (df['Day'] == 'Friday') & (df['Day'].shift(1) == 'Friday')
  df = df.drop(index = df[cond].index)

  df['Open_fri'] = df['Open'].shift(-1)

  df = df[df['Day']=='Monday']

  df = df[['Open','Open_fri']]

  df.columns = ['buy_at','sell_at']

  df['return'] = df['sell_at'] / df['buy_at']

  cond = (df['sell_at'] != 0) & (df['buy_at'] != 0)
  df = df[cond]

  return df[['return']].cumprod().iloc[-1,-1] - 1

In [ ]:
get_return_mf('005930')

In [ ]:
!pip install dart-fss

In [ ]:
# 상장 종목 가져오기

import dart_fss as dart_fss
import pandas as pd

api_key = ''
dart_fss.set_api_key(api_key=api_key)

all = dart_fss.api.filings.get_corp_code()

df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]

In [ ]:
# 상장기업 10개 랜덤 추출 후 MF_Trading 최적화 종목 찾기

df = pd.DataFrame()
for row in df_listed.sample(10)[['stock_code','corp_name']].itertuples():
  try:
    doc = {
        'name' : row[2],
        'return' : get_return_mf(row[1])
    }
    df = df.append(doc, ignore_index = True)
  except:
    print(f'error - {row[2]}')

df.sort_values(by='return', ascending=False)

In [ ]:
# 상장기업 M_F Trading 최적화 종목 찾기

df = pd.DataFrame()
for row in df_listed[['stock_code','corp_name']].itertuples():
  try:
    doc = {
        'name' : row[2],
        'return' : get_return_mf(row[1])
    }
    df = df.append(doc, ignore_index = True)
  except:
    print(f'error - {row[2]}')

df.sort_values(by='return', ascending=False)